# Research Environment Experiments

This notebook demonstrates how to use the research environment and agents to run experiments, evaluate performance and accuracy, and generate comparison plots.  It mirrors the functionality of the `run_experiments.py` script supplied in the project.


In [ ]:

# Import the research environment and agents
from research_env import ResearchEnv
from agents import QLearningAgent, SFAgent, BAMDPSRAgent
from experiment import run_experiment, evaluate_accuracy, plot_metrics

# Create a drifting environment (context may switch once per episode)
env = ResearchEnv(n_states=6, n_actions=2, drift_in_episode=True)

# Training parameters
n_episodes = 200
max_steps = 20


In [ ]:

# Train Q-learning agent
results = {}
results['Q-learning'] = run_experiment(env, QLearningAgent, n_episodes, max_steps, {
    'alpha': 0.1, 'epsilon': 0.1, 'gamma': 0.95
})

# Train successor-feature agent
results['Successor Features'] = run_experiment(env, SFAgent, n_episodes, max_steps, {
    'alpha_sr': 0.3, 'alpha_w': 0.3, 'gamma': 0.95, 'epsilon': 0.1
})

# Train adaptive SR agent with drift controller
results['Adaptive SR'] = run_experiment(env, BAMDPSRAgent, n_episodes, max_steps, {
    'alpha_sr': 0.2, 'alpha_w': 0.2, 'alpha_t': 0.05,
    'gamma': 0.95, 'drift_alpha': 1.0, 'drift_beta': 1.0,
    'drift_threshold': 0.5, 'explore_steps': 3
})

print("Training complete.")


In [ ]:

# Evaluate accuracy and average returns on a static environment (no drift)
eval_env = ResearchEnv(n_states=6, n_actions=2, drift_in_episode=False)
accuracies = {}
for name, res in results.items():
    agent = res['agent']
    avg_ret = agent.evaluate(n_episodes=30, max_steps=max_steps)
    acc = evaluate_accuracy(eval_env, agent, n_episodes=30, max_steps=max_steps)
    accuracies[name] = acc
    print(f"{name}: average return = {avg_ret:.3f}, accuracy = {acc:.3f}")

# Plot metrics (smoothed returns, training times and accuracies)
plot_metrics(results, accuracies, smoothing=10)

print("Comparison plot saved as comparison_metrics.png")


In [ ]:

# To interactively explore the environment, uncomment and run the following line.
# Note: this requires input from the user in each cell.
# from experiment import interactive_env_demo
# interactive_env_demo(env, n_steps=10)
